In [ ]:
storage_account = 'steduanalytics__update_this'
use_test_env = True

In [ ]:
if use_test_env:
    stage1 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1'
    stage2 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2'
    stage3 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage3'
else:
    stage1 = 'abfss://stage1@' + storage_account + '.dfs.core.windows.net'
    stage2 = 'abfss://stage2@' + storage_account + '.dfs.core.windows.net'
    stage3 = 'abfss://stage3@' + storage_account + '.dfs.core.windows.net'

In [ ]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the data lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path, tables):
    spark.sql('CREATE DATABASE IF NOT EXISTS ' + db_name)
    for table in tables:
        spark.sql("create table if not exists " + db_name + "." + table + " using PARQUET location '" + source_path + "/" + table + "'")

tables = ['AbsenceEventCategoryDescriptor',
    'AcademicHonorCategoryDescriptor',
    'AcademicSubjectDescriptor',
    'AcademicWeek',
    'AccommodationDescriptor',
    'Account',
    'AccountabilityRating',
    'AccountAccountCode',
    'AccountClassificationDescriptor',
    'AccountCode',
    'AchievementCategoryDescriptor',
    'Actual',
    'AdditionalCreditTypeDescriptor',
    'AddressTypeDescriptor',
    'AdministrationEnvironmentDescriptor',
    'AdministrativeFundingControlDescriptor',
    'Assessment',
    'AssessmentAcademicSubject',
    'AssessmentAssessedGradeLevel',
    'AssessmentCategoryDescriptor',
    'AssessmentContentStandard',
    'AssessmentContentStandardAuthor',
    'AssessmentIdentificationCode',
    'AssessmentIdentificationSystemDescriptor',
    'AssessmentItem',
    'AssessmentItemCategoryDescriptor',
    'AssessmentItemLearningStandard',
    'AssessmentItemPossibleResponse',
    'AssessmentItemResultDescriptor',
    'AssessmentLanguage',
    'AssessmentPerformanceLevel',
    'AssessmentPeriod',
    'AssessmentPeriodDescriptor',
    'AssessmentPlatformType',
    'AssessmentProgram',
    'AssessmentReportingMethodDescriptor',
    'AssessmentScore',
    'AssessmentSection',
    'AttemptStatusDescriptor',
    'AttendanceEventCategoryDescriptor',
    'BehaviorDescriptor',
    'BellSchedule',
    'BellScheduleClassPeriod',
    'BellScheduleDate',
    'BellScheduleGradeLevel',
    'Budget',
    'Calendar',
    'CalendarDate',
    'CalendarDateCalendarEvent',
    'CalendarEventDescriptor',
    'CalendarGradeLevel',
    'CalendarTypeDescriptor',
    'CareerPathwayDescriptor',
    'CharterApprovalAgencyTypeDescriptor',
    'CharterStatusDescriptor',
    'CitizenshipStatusDescriptor',
    'ClassPeriod',
    'ClassPeriodMeetingTime',
    'ClassroomPositionDescriptor',
    'Cohort',
    'CohortProgram',
    'CohortScopeDescriptor',
    'CohortTypeDescriptor',
    'CohortYearTypeDescriptor',
    'CommunityOrganization',
    'CommunityProvider',
    'CommunityProviderLicense',
    'CompetencyLevelDescriptor',
    'CompetencyObjective',
    'ContactTypeDescriptor',
    'ContentClassDescriptor',
    'ContinuationOfServicesReasonDescriptor',
    'ContractedStaff',
    'CostRateDescriptor',
    'CountryDescriptor',
    'Course',
    'CourseAttemptResultDescriptor',
    'CourseCompetencyLevel',
    'CourseDefinedByDescriptor',
    'CourseGPAApplicabilityDescriptor',
    'CourseIdentificationCode',
    'CourseIdentificationSystemDescriptor',
    'CourseLearningObjective',
    'CourseLearningStandard',
    'CourseLevelCharacteristic',
    'CourseLevelCharacteristicDescriptor',
    'CourseOfferedGradeLevel',
    'CourseOffering',
    'CourseOfferingCourseLevelCharacteristic',
    'CourseOfferingCurriculumUsed',
    'CourseOfferingOfferedGradeLevel',
    'CourseRepeatCodeDescriptor',
    'CourseTranscript',
    'CourseTranscriptAcademicSubject',
    'CourseTranscriptAlternativeCourseIdentificationCode',
    'CourseTranscriptCreditCategory',
    'CourseTranscriptEarnedAdditionalCredits',
    'Credential',
    'CredentialAcademicSubject',
    'CredentialEndorsement',
    'CredentialFieldDescriptor',
    'CredentialGradeLevel',
    'CredentialTypeDescriptor',
    'CreditCategoryDescriptor',
    'CreditTypeDescriptor',
    'CTEProgramServiceDescriptor',
    'CurriculumUsedDescriptor',
    'DeliveryMethodDescriptor',
    'Descriptor',
    'DiagnosisDescriptor',
    'DiplomaLevelDescriptor',
    'DiplomaTypeDescriptor',
    'DisabilityDescriptor',
    'DisabilityDesignationDescriptor',
    'DisabilityDeterminationSourceTypeDescriptor',
    'DisciplineAction',
    'DisciplineActionDiscipline',
    'DisciplineActionLengthDifferenceReasonDescriptor',
    'DisciplineActionStaff',
    'DisciplineActionStudentDisciplineIncidentAssociation',
    'DisciplineDescriptor',
    'DisciplineIncident',
    'DisciplineIncidentBehavior',
    'DisciplineIncidentExternalParticipant',
    'DisciplineIncidentParticipationCodeDescriptor',
    'DisciplineIncidentWeapon',
    'EducationalEnvironmentDescriptor',
    'EducationContent',
    'EducationContentAppropriateGradeLevel',
    'EducationContentAppropriateSex',
    'EducationContentAuthor',
    'EducationContentDerivativeSourceEducationContent',
    'EducationContentDerivativeSourceLearningResourceMetadataURI',
    'EducationContentDerivativeSourceURI',
    'EducationContentLanguage',
    'EducationOrganization',
    'EducationOrganizationAddress',
    'EducationOrganizationAddressPeriod',
    'EducationOrganizationCategory',
    'EducationOrganizationCategoryDescriptor',
    'EducationOrganizationIdentificationCode',
    'EducationOrganizationIdentificationSystemDescriptor',
    'EducationOrganizationIndicator',
    'EducationOrganizationIndicatorPeriod',
    'EducationOrganizationInstitutionTelephone',
    'EducationOrganizationInternationalAddress',
    'EducationOrganizationInterventionPrescriptionAssociation',
    'EducationOrganizationNetwork',
    'EducationOrganizationNetworkAssociation',
    'EducationOrganizationPeerAssociation',
    'EducationPlanDescriptor',
    'EducationServiceCenter',
    'ElectronicMailTypeDescriptor',
    'EmploymentStatusDescriptor',
    'EntryGradeLevelReasonDescriptor',
    'EntryTypeDescriptor',
    'EventCircumstanceDescriptor',
    'ExitWithdrawTypeDescriptor',
    'FeederSchoolAssociation',
    'GeneralStudentProgramAssociation',
    'GeneralStudentProgramAssociationParticipationStatus',
    'Grade',
    'GradebookEntry',
    'GradebookEntryLearningObjective',
    'GradebookEntryLearningStandard',
    'GradebookEntryTypeDescriptor',
    'GradeLearningStandardGrade',
    'GradeLevelDescriptor',
    'GradePointAverageTypeDescriptor',
    'GradeTypeDescriptor',
    'GradingPeriod',
    'GradingPeriodDescriptor',
    'GraduationPlan',
    'GraduationPlanCreditsByCourse',
    'GraduationPlanCreditsByCourseCourse',
    'GraduationPlanCreditsByCreditCategory',
    'GraduationPlanCreditsBySubject',
    'GraduationPlanRequiredAssessment',
    'GraduationPlanRequiredAssessmentPerformanceLevel',
    'GraduationPlanRequiredAssessmentScore',
    'GraduationPlanTypeDescriptor',
    'GunFreeSchoolsActReportingStatusDescriptor',
    'HomelessPrimaryNighttimeResidenceDescriptor',
    'HomelessProgramServiceDescriptor',
    'IdentificationDocumentUseDescriptor',
    'IncidentLocationDescriptor',
    'IndicatorDescriptor',
    'IndicatorGroupDescriptor',
    'IndicatorLevelDescriptor',
    'InstitutionTelephoneNumberTypeDescriptor',
    'InteractivityStyleDescriptor',
    'InternetAccessDescriptor',
    'Intervention',
    'InterventionAppropriateGradeLevel',
    'InterventionAppropriateSex',
    'InterventionClassDescriptor',
    'InterventionDiagnosis',
    'InterventionEducationContent',
    'InterventionEffectivenessRatingDescriptor',
    'InterventionInterventionPrescription',
    'InterventionLearningResourceMetadataURI',
    'InterventionMeetingTime',
    'InterventionPopulationServed',
    'InterventionPrescription',
    'InterventionPrescriptionAppropriateGradeLevel',
    'InterventionPrescriptionAppropriateSex',
    'InterventionPrescriptionDiagnosis',
    'InterventionPrescriptionEducationContent',
    'InterventionPrescriptionLearningResourceMetadataURI',
    'InterventionPrescriptionPopulationServed',
    'InterventionPrescriptionURI',
    'InterventionStaff',
    'InterventionStudy',
    'InterventionStudyAppropriateGradeLevel',
    'InterventionStudyAppropriateSex',
    'InterventionStudyEducationContent',
    'InterventionStudyInterventionEffectiveness',
    'InterventionStudyLearningResourceMetadataURI',
    'InterventionStudyPopulationServed',
    'InterventionStudyStateAbbreviation',
    'InterventionStudyURI',
    'InterventionURI',
    'LanguageDescriptor',
    'LanguageInstructionProgramServiceDescriptor',
    'LanguageUseDescriptor',
    'LearningObjective',
    'LearningObjectiveAcademicSubject',
    'LearningObjectiveContentStandard',
    'LearningObjectiveContentStandardAuthor',
    'LearningObjectiveGradeLevel',
    'LearningObjectiveLearningStandard',
    'LearningStandard',
    'LearningStandardAcademicSubject',
    'LearningStandardCategoryDescriptor',
    'LearningStandardContentStandard',
    'LearningStandardContentStandardAuthor',
    'LearningStandardEquivalenceAssociation',
    'LearningStandardEquivalenceStrengthDescriptor',
    'LearningStandardGradeLevel',
    'LearningStandardIdentificationCode',
    'LearningStandardPrerequisiteLearningStandard',
    'LearningStandardScopeDescriptor',
    'LevelOfEducationDescriptor',
    'LicenseStatusDescriptor',
    'LicenseTypeDescriptor',
    'LimitedEnglishProficiencyDescriptor',
    'LocaleDescriptor',
    'LocalEducationAgency',
    'LocalEducationAgencyAccountability',
    'LocalEducationAgencyCategoryDescriptor',
    'LocalEducationAgencyFederalFunds',
    'Location',
    'MagnetSpecialProgramEmphasisSchoolDescriptor',
    'MediumOfInstructionDescriptor',
    'MethodCreditEarnedDescriptor',
    'MigrantEducationProgramServiceDescriptor',
    'MonitoredDescriptor',
    'NeglectedOrDelinquentProgramDescriptor',
    'NeglectedOrDelinquentProgramServiceDescriptor',
    'NetworkPurposeDescriptor',
    'ObjectiveAssessment',
    'ObjectiveAssessmentAssessmentItem',
    'ObjectiveAssessmentLearningObjective',
    'ObjectiveAssessmentLearningStandard',
    'ObjectiveAssessmentPerformanceLevel',
    'ObjectiveAssessmentScore',
    'OldEthnicityDescriptor',
    'OpenStaffPosition',
    'OpenStaffPositionAcademicSubject',
    'OpenStaffPositionInstructionalGradeLevel',
    'OperationalStatusDescriptor',
    'OtherNameTypeDescriptor',
    'Parent',
    'ParentAddress',
    'ParentAddressPeriod',
    'ParentElectronicMail',
    'ParentInternationalAddress',
    'ParentLanguage',
    'ParentLanguageUse',
    'ParentOtherName',
    'ParentPersonalIdentificationDocument',
    'ParentTelephone',
    'ParticipationDescriptor',
    'ParticipationStatusDescriptor',
    'Payroll',
    'PerformanceBaseConversionDescriptor',
    'PerformanceLevelDescriptor',
    'Person',
    'PersonalInformationVerificationDescriptor',
    'PlatformTypeDescriptor',
    'PopulationServedDescriptor',
    'PostingResultDescriptor',
    'PostSecondaryEvent',
    'PostSecondaryEventCategoryDescriptor',
    'PostSecondaryInstitution',
    'PostSecondaryInstitutionLevelDescriptor',
    'PostSecondaryInstitutionMediumOfInstruction',
    'ProficiencyDescriptor',
    'Program',
    'ProgramAssignmentDescriptor',
    'ProgramCharacteristic',
    'ProgramCharacteristicDescriptor',
    'ProgramLearningObjective',
    'ProgramLearningStandard',
    'ProgramService',
    'ProgramSponsor',
    'ProgramSponsorDescriptor',
    'ProgramTypeDescriptor',
    'ProgressDescriptor',
    'ProgressLevelDescriptor',
    'ProviderCategoryDescriptor',
    'ProviderProfitabilityDescriptor',
    'ProviderStatusDescriptor',
    'PublicationStatusDescriptor',
    'QuestionFormDescriptor',
    'RaceDescriptor',
    'ReasonExitedDescriptor',
    'ReasonNotTestedDescriptor',
    'RecognitionTypeDescriptor',
    'RelationDescriptor',
    'RepeatIdentifierDescriptor',
    'ReportCard',
    'ReportCardGrade',
    'ReportCardGradePointAverage',
    'ReportCardStudentCompetencyObjective',
    'ReportCardStudentLearningObjective',
    'ReporterDescriptionDescriptor',
    'ResidencyStatusDescriptor',
    'ResponseIndicatorDescriptor',
    'ResponsibilityDescriptor',
    'RestraintEvent',
    'RestraintEventProgram',
    'RestraintEventReason',
    'RestraintEventReasonDescriptor',
    'ResultDatatypeTypeDescriptor',
    'RetestIndicatorDescriptor',
    'School',
    'SchoolCategory',
    'SchoolCategoryDescriptor',
    'SchoolChoiceImplementStatusDescriptor',
    'SchoolFoodServiceProgramServiceDescriptor',
    'SchoolGradeLevel',
    'SchoolTypeDescriptor',
    'SchoolYearType',
    'Section',
    'SectionAttendanceTakenEvent',
    'SectionCharacteristic',
    'SectionCharacteristicDescriptor',
    'SectionClassPeriod',
    'SectionCourseLevelCharacteristic',
    'SectionOfferedGradeLevel',
    'SectionProgram',
    'SeparationDescriptor',
    'SeparationReasonDescriptor',
    'ServiceDescriptor',
    'Session',
    'SessionAcademicWeek',
    'SessionGradingPeriod',
    'SexDescriptor',
    'SourceSystemDescriptor',
    'SpecialEducationProgramServiceDescriptor',
    'SpecialEducationSettingDescriptor',
    'Staff',
    'StaffAbsenceEvent',
    'StaffAddress',
    'StaffAddressPeriod',
    'StaffClassificationDescriptor',
    'StaffCohortAssociation',
    'StaffCredential',
    'StaffDisciplineIncidentAssociation',
    'StaffDisciplineIncidentAssociationDisciplineIncidentParticipationCode',
    'StaffEducationOrganizationAssignmentAssociation',
    'StaffEducationOrganizationContactAssociation',
    'StaffEducationOrganizationContactAssociationAddress',
    'StaffEducationOrganizationContactAssociationAddressPeriod',
    'StaffEducationOrganizationContactAssociationTelephone',
    'StaffEducationOrganizationEmploymentAssociation',
    'StaffElectronicMail',
    'StaffIdentificationCode',
    'StaffIdentificationDocument',
    'StaffIdentificationSystemDescriptor',
    'StaffInternationalAddress',
    'StaffLanguage',
    'StaffLanguageUse',
    'StaffLeave',
    'StaffLeaveEventCategoryDescriptor',
    'StaffOtherName',
    'StaffPersonalIdentificationDocument',
    'StaffProgramAssociation',
    'StaffRace',
    'StaffRecognition',
    'StaffSchoolAssociation',
    'StaffSchoolAssociationAcademicSubject',
    'StaffSchoolAssociationGradeLevel',
    'StaffSectionAssociation',
    'StaffTelephone',
    'StaffTribalAffiliation',
    'StaffVisa',
    'StateAbbreviationDescriptor',
    'StateEducationAgency',
    'StateEducationAgencyAccountability',
    'StateEducationAgencyFederalFunds',
    'Student',
    'StudentAcademicRecord',
    'StudentAcademicRecordAcademicHonor',
    'StudentAcademicRecordClassRanking',
    'StudentAcademicRecordDiploma',
    'StudentAcademicRecordGradePointAverage',
    'StudentAcademicRecordRecognition',
    'StudentAcademicRecordReportCard',
    'StudentAssessment',
    'StudentAssessmentAccommodation',
    'StudentAssessmentItem',
    'StudentAssessmentPerformanceLevel',
    'StudentAssessmentScoreResult',
    'StudentAssessmentStudentObjectiveAssessment',
    'StudentAssessmentStudentObjectiveAssessmentPerformanceLevel',
    'StudentAssessmentStudentObjectiveAssessmentScoreResult',
    'StudentCharacteristicDescriptor',
    'StudentCohortAssociation',
    'StudentCohortAssociationSection',
    'StudentCompetencyObjective',
    'StudentCompetencyObjectiveGeneralStudentProgramAssociation',
    'StudentCompetencyObjectiveStudentSectionAssociation',
    'StudentCTEProgramAssociation',
    'StudentCTEProgramAssociationCTEProgram',
    'StudentCTEProgramAssociationCTEProgramService',
    'StudentCTEProgramAssociationService',
    'StudentDisciplineIncidentAssociation',
    'StudentDisciplineIncidentAssociationBehavior',
    'StudentEducationOrganizationAssociation',
    'StudentEducationOrganizationAssociationAddress',
    'StudentEducationOrganizationAssociationAddressPeriod',
    'StudentEducationOrganizationAssociationCohortYear',
    'StudentEducationOrganizationAssociationDisability',
    'StudentEducationOrganizationAssociationDisabilityDesignation',
    'StudentEducationOrganizationAssociationElectronicMail',
    'StudentEducationOrganizationAssociationInternationalAddress',
    'StudentEducationOrganizationAssociationLanguage',
    'StudentEducationOrganizationAssociationLanguageUse',
    'StudentEducationOrganizationAssociationProgramParticipation',
    'StudentEducationOrganizationAssociationProgramParticipationProgramCharacteristic',
    'StudentEducationOrganizationAssociationRace',
    'StudentEducationOrganizationAssociationStudentCharacteristic',
    'StudentEducationOrganizationAssociationStudentCharacteristicPeriod',
    'StudentEducationOrganizationAssociationStudentIdentificationCode',
    'StudentEducationOrganizationAssociationStudentIndicator',
    'StudentEducationOrganizationAssociationStudentIndicatorPeriod',
    'StudentEducationOrganizationAssociationTelephone',
    'StudentEducationOrganizationAssociationTribalAffiliation',
    'StudentEducationOrganizationResponsibilityAssociation',
    'StudentGradebookEntry',
    'StudentHomelessProgramAssociation',
    'StudentHomelessProgramAssociationHomelessProgramService',
    'StudentIdentificationDocument',
    'StudentIdentificationSystemDescriptor',
    'StudentInterventionAssociation',
    'StudentInterventionAssociationInterventionEffectiveness',
    'StudentInterventionAttendanceEvent',
    'StudentLanguageInstructionProgramAssociation',
    'StudentLanguageInstructionProgramAssociationEnglishLanguageProficiencyAssessment',
    'StudentLanguageInstructionProgramAssociationLanguageInstructionProgramService',
    'StudentLearningObjective',
    'StudentLearningObjectiveGeneralStudentProgramAssociation',
    'StudentLearningObjectiveStudentSectionAssociation',
    'StudentMigrantEducationProgramAssociation',
    'StudentMigrantEducationProgramAssociationMigrantEducationProgramService',
    'StudentNeglectedOrDelinquentProgramAssociation',
    'StudentNeglectedOrDelinquentProgramAssociationNeglectedOrDelinquentProgramService',
    'StudentOtherName',
    'StudentParentAssociation',
    'StudentParticipationCodeDescriptor',
    'StudentPersonalIdentificationDocument',
    'StudentProgramAssociation',
    'StudentProgramAssociationService',
    'StudentProgramAttendanceEvent',
    'StudentSchoolAssociation',
    'StudentSchoolAssociationAlternativeGraduationPlan',
    'StudentSchoolAssociationEducationPlan',
    'StudentSchoolAttendanceEvent',
    'StudentSchoolFoodServiceProgramAssociation',
    'StudentSchoolFoodServiceProgramAssociationSchoolFoodServiceProgramService',
    'StudentSectionAssociation',
    'StudentSectionAttendanceEvent',
    'StudentSpecialEducationProgramAssociation',
    'StudentSpecialEducationProgramAssociationDisability',
    'StudentSpecialEducationProgramAssociationDisabilityDesignation',
    'StudentSpecialEducationProgramAssociationServiceProvider',
    'StudentSpecialEducationProgramAssociationSpecialEducationProgramService',
    'StudentSpecialEducationProgramAssociationSpecialEducationProgramServiceProvider',
    'StudentTitleIPartAProgramAssociation',
    'StudentTitleIPartAProgramAssociationService',
    'StudentTitleIPartAProgramAssociationTitleIPartAProgramService',
    'StudentVisa',
    'Survey',
    'SurveyCategoryDescriptor',
    'SurveyCourseAssociation',
    'SurveyLevelDescriptor',
    'SurveyProgramAssociation',
    'SurveyQuestion',
    'SurveyQuestionMatrix',
    'SurveyQuestionResponse',
    'SurveyQuestionResponseChoice',
    'SurveyQuestionResponseSurveyQuestionMatrixElementResponse',
    'SurveyQuestionResponseValue',
    'SurveyResponse',
    'SurveyResponseEducationOrganizationTargetAssociation',
    'SurveyResponseStaffTargetAssociation',
    'SurveyResponseSurveyLevel',
    'SurveySection',
    'SurveySectionAssociation',
    'SurveySectionResponse',
    'SurveySectionResponseEducationOrganizationTargetAssociation',
    'SurveySectionResponseStaffTargetAssociation',
    'TeachingCredentialBasisDescriptor',
    'TeachingCredentialDescriptor',
    'TechnicalSkillsAssessmentDescriptor',
    'TelephoneNumberTypeDescriptor',
    'TermDescriptor',
    'TitleIPartAParticipantDescriptor',
    'TitleIPartAProgramServiceDescriptor',
    'TitleIPartASchoolDesignationDescriptor',
    'TribalAffiliationDescriptor',
    'VisaDescriptor',
    'WeaponDescriptor']

db_prefix = 'test_' if use_test_env else ''
create_spark_db(db_prefix + 's2_edfi', stage2 + '/ed-fi', tables)